In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False,
                                               split_noise=True, n_fabricate_noise = 1600,
                                               split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
/usr/local/anaconda3/lib/python3.5/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


# Train
* Hmm, sparse matrix doesnt work

In [3]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_multi_lstm_random_input_modify import *

In [4]:
graph_name = 'SpectrogramMultiLSTMRandomInputModify'  # Specify graph class name

g_cnfg = Config('graph_02')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[1].shape[1]

g_cnfg.random_modify_brightness_max_delta = 0.25
g_cnfg.random_modify_contrast_lower = 0.2
g_cnfg.random_modify_contrast_upper = 1.8

#g_cnfg.conv1_n_filters = 16
#g_cnfg.conv1_kernel_size = 5
#g_cnfg.conv1_mp_size = 5
#g_cnfg.conv1_mp_strides = 2

#g_cnfg.conv2_n_filters = 32
#g_cnfg.conv2_kernel_size = 3
#g_cnfg.conv2_mp_size = 3
#g_cnfg.conv2_mp_strides = 2

g_cnfg.lstm_state_sizes = [100, 200]
#g_cnfg.lstm_dropout = {'in': 'y', 'out': 'n', 'state': 'n'}

#g_cnfg.X_conv_h = math.ceil(math.ceil(g_cnfg.X_img_h / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
#g_cnfg.X_conv_w = math.ceil(math.ceil(g_cnfg.X_img_w / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
#g_cnfg.n_conv_flat = g_cnfg.X_conv_h * g_cnfg.X_conv_w * g_cnfg.conv2_n_filters
#g_cnfg.n_flat = g_cnfg.n_conv_flat + g_cnfg.lstm_state_sizes[-1]

g_cnfg.n_hidden1 = 50
#g_cnfg.n_hidden2 = 50

g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 0.7  # 1.0 means no decay

t_cnfg = Config('run_01')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.add_noise = (0.5, [s.spec_data for s in train.get_list_of_label('_background_noise_')])
t_cnfg.dropout_keep_prob = 0.75
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0]/t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [5]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_02
X_img_h: 128
X_img_w: 71
Y_vector_len: 31
lr_decay_rate: 0.7
lr_decay_steps: 1,000,000
lr_initial: 0.1
lstm_state_sizes: [100, 200]
n_hidden1: 50
random_modify_brightness_max_delta: 0.25
random_modify_contrast_lower: 0.2
random_modify_contrast_upper: 1.8

name: run_01
add_noise: (0.5, 'list of length 2,002')
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.75
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 379
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

SpectrogramMultiLSTMRandomInputModify_graph_02_run_01
Training starts @ 12/17/2017 15:59:20
Epoch 1 Step 1 Best model saved @ 12/17/2017 15:59:25 [Ave valid] 3.434
Epoch 1 Step 1 ends @ 12/17/2017 15:59:25 [Train] 3.434, 4.0% [Valid] 2.8% [Ave valid] 3.434
Epoch 1 Step 50 Best model saved @ 12/17/2017 15:59:35 [Ave valid] 3.434
Epoch 1 Step 100 Best model saved @ 12/17/2017 15:59:45 [Ave valid] 3.433
Epoch 1 Step 250 

Epoch 14 Step 4,950 Best model saved @ 12/17/2017 16:16:37 [Ave valid] 0.378
Epoch 14 Step 5,000 Best model saved @ 12/17/2017 16:16:47 [Ave valid] 0.373
Epoch 14 Step 5,050 Best model saved @ 12/17/2017 16:16:57 [Ave valid] 0.369
Epoch 14 Step 5,200 Best model saved @ 12/17/2017 16:17:28 [Ave valid] 0.369
Epoch 14 Step 5,250 Best model saved @ 12/17/2017 16:17:38 [Ave valid] 0.367
Epoch 14 Step 5,300 Best model saved @ 12/17/2017 16:17:48 [Ave valid] 0.363
Epoch 14 Step 5,306 ends @ 12/17/2017 16:17:52 [Train] 0.130, 100.0% [Valid] 84.8% [Ave valid] 0.392
Epoch 15 Step 5,685 Best model saved @ 12/17/2017 16:19:10 [Ave valid] 0.348
Epoch 15 Step 5,685 ends @ 12/17/2017 16:19:10 [Train] 0.231, 92.9% [Valid] 88.8% [Ave valid] 0.348
Epoch 16 Step 5,700 Best model saved @ 12/17/2017 16:19:16 [Ave valid] 0.346
Epoch 16 Step 5,750 Best model saved @ 12/17/2017 16:19:26 [Ave valid] 0.346
Epoch 16 Step 5,900 Best model saved @ 12/17/2017 16:19:55 [Ave valid] 0.340
Epoch 16 Step 5,950 Best mode

Epoch 38 Step 14,400 Best model saved @ 12/17/2017 16:49:09 [Ave valid] 0.202
Epoch 38 Step 14,402 ends @ 12/17/2017 16:49:13 [Train] 0.002, 100.0% [Valid] 94.1% [Ave valid] 0.204
Epoch 39 Step 14,750 Best model saved @ 12/17/2017 16:50:22 [Ave valid] 0.202
Epoch 39 Step 14,781 ends @ 12/17/2017 16:50:29 [Train] 0.103, 100.0% [Valid] 93.8% [Ave valid] 0.202
Epoch 40 Step 14,850 Best model saved @ 12/17/2017 16:50:45 [Ave valid] 0.201
Epoch 40 Step 15,000 Best model saved @ 12/17/2017 16:51:15 [Ave valid] 0.201
Epoch 40 Step 15,150 Best model saved @ 12/17/2017 16:51:44 [Ave valid] 0.198
Epoch 40 Step 15,160 ends @ 12/17/2017 16:51:48 [Train] 0.159, 92.9% [Valid] 92.1% [Ave valid] 0.209
Epoch 41 Step 15,539 ends @ 12/17/2017 16:53:04 [Train] 0.020, 100.0% [Valid] 94.7% [Ave valid] 0.199
Epoch 42 Step 15,650 Best model saved @ 12/17/2017 16:53:30 [Ave valid] 0.198
Epoch 42 Step 15,700 Best model saved @ 12/17/2017 16:53:39 [Ave valid] 0.197
Epoch 42 Step 15,750 Best model saved @ 12/17/2

Epoch 73 Step 27,650 Best model saved @ 12/17/2017 17:34:08 [Ave valid] 0.161
Epoch 73 Step 27,667 ends @ 12/17/2017 17:34:13 [Train] 0.015, 100.0% [Valid] 95.3% [Ave valid] 0.163
Epoch 74 Step 28,046 ends @ 12/17/2017 17:35:28 [Train] 0.005, 100.0% [Valid] 95.7% [Ave valid] 0.169
Epoch 75 Step 28,425 ends @ 12/17/2017 17:36:45 [Train] 0.001, 100.0% [Valid] 95.7% [Ave valid] 0.164
Epoch 76 Step 28,804 ends @ 12/17/2017 17:38:03 [Train] 0.004, 100.0% [Valid] 94.9% [Ave valid] 0.167
Epoch 77 Step 29,183 ends @ 12/17/2017 17:39:18 [Train] 0.004, 100.0% [Valid] 91.6% [Ave valid] 0.182
Epoch 78 Step 29,562 ends @ 12/17/2017 17:40:36 [Train] 0.190, 92.9% [Valid] 92.8% [Ave valid] 0.178
Epoch 79 Step 29,941 ends @ 12/17/2017 17:41:51 [Train] 0.018, 100.0% [Valid] 95.5% [Ave valid] 0.164
Epoch 80 Step 30,320 Best model saved @ 12/17/2017 17:43:09 [Ave valid] 0.160
Epoch 80 Step 30,320 ends @ 12/17/2017 17:43:09 [Train] 0.172, 92.9% [Valid] 95.9% [Ave valid] 0.160
Epoch 81 Step 30,550 Best mode

Epoch 129 Step 48,891 ends @ 12/17/2017 18:44:53 [Train] 0.165, 92.9% [Valid] 96.3% [Ave valid] 0.149
Epoch 130 Step 49,250 Best model saved @ 12/17/2017 18:46:05 [Ave valid] 0.145
Epoch 130 Step 49,270 ends @ 12/17/2017 18:46:10 [Train] 0.001, 100.0% [Valid] 96.2% [Ave valid] 0.145
Epoch 131 Step 49,300 Best model saved @ 12/17/2017 18:46:18 [Ave valid] 0.145
Epoch 131 Step 49,649 ends @ 12/17/2017 18:47:24 [Train] 0.045, 100.0% [Valid] 93.9% [Ave valid] 0.159
Epoch 132 Step 50,028 ends @ 12/17/2017 18:48:40 [Train] 0.002, 100.0% [Valid] 96.2% [Ave valid] 0.148
Epoch 133 Step 50,407 ends @ 12/17/2017 18:49:56 [Train] 0.002, 100.0% [Valid] 96.2% [Ave valid] 0.150
Epoch 134 Step 50,786 ends @ 12/17/2017 18:51:10 [Train] 0.000, 100.0% [Valid] 96.3% [Ave valid] 0.147
Epoch 135 Step 51,165 ends @ 12/17/2017 18:52:26 [Train] 0.001, 100.0% [Valid] 96.1% [Ave valid] 0.145
Epoch 136 Step 51,544 ends @ 12/17/2017 18:53:40 [Train] 0.004, 100.0% [Valid] 96.0% [Ave valid] 0.147
Epoch 137 Step 51,9

Epoch 199 Step 75,421 ends @ 12/17/2017 20:12:44 [Train] 0.000, 100.0% [Valid] 96.5% [Ave valid] 0.148
Epoch 200 Step 75,800 ends @ 12/17/2017 20:13:57 [Train] 0.005, 100.0% [Valid] 96.4% [Ave valid] 0.146
Epoch 201 Step 76,179 ends @ 12/17/2017 20:15:11 [Train] 0.001, 100.0% [Valid] 96.5% [Ave valid] 0.145
Epoch 202 Step 76,558 ends @ 12/17/2017 20:16:28 [Train] 0.009, 100.0% [Valid] 96.2% [Ave valid] 0.149
Epoch 203 Step 76,937 ends @ 12/17/2017 20:17:42 [Train] 0.297, 92.9% [Valid] 95.9% [Ave valid] 0.152
Epoch 204 Step 77,316 ends @ 12/17/2017 20:18:58 [Train] 0.000, 100.0% [Valid] 96.5% [Ave valid] 0.147
Epoch 205 Step 77,695 ends @ 12/17/2017 20:20:12 [Train] 0.000, 100.0% [Valid] 96.3% [Ave valid] 0.150
Epoch 206 Step 78,074 ends @ 12/17/2017 20:21:28 [Train] 1.463, 78.6% [Valid] 93.1% [Ave valid] 0.170
Epoch 207 Step 78,453 ends @ 12/17/2017 20:22:45 [Train] 0.103, 100.0% [Valid] 95.7% [Ave valid] 0.152
Epoch 208 Step 78,832 ends @ 12/17/2017 20:23:59 [Train] 0.003, 100.0% [Val

Epoch 275 Step 104,225 ends @ 12/17/2017 21:48:13 [Train] 0.135, 100.0% [Valid] 96.6% [Ave valid] 0.153
Epoch 276 Step 104,604 ends @ 12/17/2017 21:49:30 [Train] 0.005, 100.0% [Valid] 95.9% [Ave valid] 0.156
Epoch 277 Step 104,983 ends @ 12/17/2017 21:50:44 [Train] 0.001, 100.0% [Valid] 96.5% [Ave valid] 0.149
Epoch 278 Step 105,362 ends @ 12/17/2017 21:52:00 [Train] 0.000, 100.0% [Valid] 96.6% [Ave valid] 0.150
Epoch 279 Step 105,741 ends @ 12/17/2017 21:53:14 [Train] 0.003, 100.0% [Valid] 96.3% [Ave valid] 0.149
Epoch 280 Step 106,120 ends @ 12/17/2017 21:54:31 [Train] 0.003, 100.0% [Valid] 96.2% [Ave valid] 0.153
Epoch 281 Step 106,499 ends @ 12/17/2017 21:55:45 [Train] 0.000, 100.0% [Valid] 96.4% [Ave valid] 0.152
Epoch 282 Step 106,878 ends @ 12/17/2017 21:57:02 [Train] 0.015, 100.0% [Valid] 94.9% [Ave valid] 0.162
Epoch 283 Step 107,257 ends @ 12/17/2017 21:58:19 [Train] 0.001, 100.0% [Valid] 96.5% [Ave valid] 0.152
Epoch 284 Step 107,636 ends @ 12/17/2017 21:59:32 [Train] 0.002,

KeyboardInterrupt: 